In [1]:
import sys
import os
import numpy as np
import random
import json
import string
import itertools
import math
from Crypto.Util.number import long_to_bytes as ltb, bytes_to_long as btl, inverse
from pwn import remote, process, log, args, debug, info, log as pwnLog
from tqdm import tqdm, trange
from matplotlib import pyplot as plt
from functools import reduce, cache
from base64 import b64encode, b64decode
from collections import Counter

## Create a look up table for first 2 bytes

In [2]:
with open('output.txt', 'r') as ofile:
    mat = eval(ofile.read())

mat = np.array(mat)
N = 1600
T = 1/800
x = np.linspace(0, N*T, N)

# Man in the Middle techniques for searching sum of 4 elements in an array that equal our target
table = {}
for a in trange(256):
    ass = 0.5 ** 0 * np.sin(a * 2 * np.pi * x)
    for b in range(256):
        bss = ass + 0.5 * np.sin(b * 2 * np.pi * x)
        table[tuple(bss.round(10))] = (a, b)
print('init done!')

100%|██████████| 256/256 [00:06<00:00, 38.55it/s]

init done!


## Bruteforcing other 2 bytes

In [3]:
key = []
for i in range(4):
    good = False
    for c in trange(256):
        ass = 0.5**2 * np.sin(c * 2 * np.pi * x)
        for d in range(256):
            bss = ass + 0.5**3 * np.sin(d * 2 * np.pi * x)
            s = tuple((mat[i]-bss).round(10))
            ab = table.get(s, None)
            if ab is not None:
                a, b = ab
                key.extend([a, b, c, d])
                good = True
                break
        if good:
            break
    else:
        raise Exception("Wrong!")
key

 50%|████▉     | 127/256 [00:02<00:02, 56.63it/s]


[241, 62, 82, 133, 193, 113, 73, 146, 144, 107, 159, 97, 241, 220, 127, 68]

## Decrypting flag

In [5]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
key = bytes(key)
with open('ciphertext.bin', 'rb') as f:
    iv = f.read(16)
    cip = AES.new(key, AES.MODE_CBC, iv=iv)
    ct = f.read()
    print(unpad(cip.decrypt(ct), 16).decode())

HCMUS-CTF{https://www.youtube.com/watch?v=nmgFG7PUHfo}
